In [1]:
import pandas as pd

Read in data

In [2]:
allergies = pd.read_csv('allergies.csv')
careplans = pd.read_csv('careplans.csv')
# claims = pd.read_csv('claims.csv')
conditions = pd.read_csv('conditions.csv')
encounters = pd.read_csv('encounters.csv')
immunizations = pd.read_csv('immunizations.csv')
medications = pd.read_csv('medications.csv')
observations = pd.read_csv('observations.csv')
patients = pd.read_csv('patients.csv')
procedures = pd.read_csv('procedures.csv')

Drop Columns

In [3]:
column_names = ['START','STOP']
allergies_clean = allergies.drop(column_names, axis=1)
careplans_clean = careplans.drop(column_names, axis=1)
conditions_clean = conditions.drop(column_names, axis=1)
medications_clean = medications.drop(column_names, axis=1)

# Drop unnecessary patient information
patients = patients.drop('ssn', axis=1)
patients = patients.drop('passport', axis=1)
patients = patients.drop('first', axis=1)
patients = patients.drop('last', axis=1)
patients = patients.drop('maiden', axis=1)
patients = patients.drop('suffix', axis=1)
patients = patients.drop('prefix', axis=1)
## Do we need address/birthplace? could be geographically interesting ig?
patients = patients.drop('drivers', axis=1) ## WHAT IS DRIVERS? don't think we need but do more research to make sure

# drop diagnosis from claims as all entries are empty
# claims = claims.drop('DIAGNOSIS', axis=1)
# claims = claims.drop('ORGANIZATION', axis=1) # deleted this as all organizations are temp organizations

column_names = ['DATE']
observations_clean = observations.drop(column_names, axis=1)
immunizations_clean = immunizations.drop(column_names, axis=1)
encounters_clean = encounters.drop(column_names, axis=1)

column_names = ['DESCRIPTION']
conditions_clean = conditions_clean.drop(column_names, axis=1)
encounters_clean = encounters_clean.drop(column_names, axis=1)
immunizations_clean = immunizations_clean.drop(column_names, axis=1)
observations_clean = observations_clean.drop(column_names, axis=1)
medications_clean = medications_clean.drop(column_names, axis=1)
procedures_clean = procedures.drop(column_names, axis=1)


medications_clean = medications_clean.drop('REASONDESCRIPTION', axis=1)
procedures_clean = procedures_clean.drop('REASONDESCRIPTION', axis=1)

Add suffixes to columns

In [4]:
patients = patients.rename(columns={'patient': 'PATIENT'})

def add_suffix(df, suffix):
    renamed_columns = {}
    for col_name in df.columns:
        if col_name != 'PATIENT':
            renamed_columns[col_name] = col_name + '_' + suffix
        else:
            renamed_columns[col_name] = col_name
    return df.rename(columns=renamed_columns)


allergies_clean = add_suffix(allergies_clean, 'ALLERGIES')
careplans_clean = add_suffix(careplans_clean, 'CAREPLANS')
# claims_clean = add_suffix(claims, 'CLAIMS')
conditions_clean = add_suffix(conditions_clean, 'CONDITIONS')
encounters_clean = add_suffix(encounters_clean, 'ENCOUNTERS')
immunizations_clean = add_suffix(immunizations_clean, 'IMMUNIZATIONS')
medications_clean = add_suffix(medications_clean, 'MEDICATIONS')
observations_clean = add_suffix(observations_clean, 'OBSERVATIONS')
procedures_clean = add_suffix(procedures_clean, 'PROCEDURES')


Split up encounters to add stuff

In [ ]:
total_rows = len(encounters_clean)
rows_per_section = total_rows // 8

section1 = encounters_clean.iloc[:rows_per_section]
section2 = encounters_clean.iloc[rows_per_section:2*rows_per_section]
section3 = encounters_clean.iloc[2*rows_per_section:3*rows_per_section]
section4 = encounters_clean.iloc[3*rows_per_section:4*rows_per_section]
section5 = encounters_clean.iloc[4*rows_per_section:5*rows_per_section]
section6 = encounters_clean.iloc[5*rows_per_section:6*rows_per_section]
section7 = encounters_clean.iloc[6*rows_per_section:7*rows_per_section]
section8 = encounters_clean.iloc[7*rows_per_section:]

section1 = add_suffix(section1, '1')
section2 = add_suffix(section2, '2')
section3 = add_suffix(section3, '3')
section4 = add_suffix(section4, '4')
section5 = add_suffix(section5, '5')
section6 = add_suffix(section6, '6')
section7 = add_suffix(section7, '7')
section8 = add_suffix(section8, '8')

Merge datasets

In [ ]:
def merge_datasets():
    
    merged_df = pd.merge(allergies_clean, patients, on='PATIENT', how='left')
    merged_df = pd.merge(merged_df, careplans_clean, on='PATIENT', how='left')
    merged_df = pd.merge(merged_df, conditions_clean, on='PATIENT', how='left')
    merged_df = pd.merge(merged_df, procedures_clean, on='PATIENT', how='left')
    
    merged_df = pd.merge(merged_df, section1, on='PATIENT', how='left')
    merged_df = pd.merge(merged_df, section2, on='PATIENT', how='left')
    merged_df = pd.merge(merged_df, section3, on='PATIENT', how='left')
    merged_df = pd.merge(merged_df, section4, on='PATIENT', how='left')
    merged_df = pd.merge(merged_df, section5, on='PATIENT', how='left')
    merged_df = pd.merge(merged_df, section6, on='PATIENT', how='left')
    merged_df = pd.merge(merged_df, section7, on='PATIENT', how='left')
    
    merged_df = pd.merge(merged_df, immunizations_clean, on='PATIENT', how='left')
    merged_df = pd.merge(merged_df, medications_clean, on='PATIENT', how='left')
    merged_df = pd.merge(merged_df, observations_clean, on='PATIENT', how='left')

Separate data based on disease

Calculate data usage

In [16]:
# # Get memory usage of each column
# memory_usage_per_column = merged_df.memory_usage(deep=True)

# # Sum up memory usage of all columns
# total_memory_usage = memory_usage_per_column.sum()

# print("Total memory usage of DataFrame: {} bytes".format(total_memory_usage))

Total memory usage of DataFrame: 20279191894 bytes


NOTES FROM PHIL TO STOP KERNEL DEATH
Size of indiv. dataframes
- delete columns that we are not going to use before we merge

- read in datasets a chunk at a time
    - operate on sequential parts of the data

- if its still an issue, could get set up on cluster to compute there as well